Data Cleaning

As stated earlier the folder structure as above 

This process is to be done on the rawdata.csv in the data/processed folder
The script wil be run from an preprocessing.ipynb file in the notebooks/002-preprocessing folder

Data Cleaning Class
Function 1 
Change the Columns into Correct Formats
1. Look for a column that could have date like features which is currently in str and change its format most probably its structured like this : date/month/year or have the name "date" as part of the name of column (if not present skip and note down in comment that there are no date features found). If found create new additional columns, year and month and fill them appropriately.

2. There are numerical columns that are refererred to as 'object', so examine each column that has more than 30% numerical values and change it to its proper format "str" and for the rest regard them as Nan or Null values. Remember that some of these are comma separated and may have spaces.

Function 2
Drop Missing Values 
1. drop all columns with missing values and Nan values of more than 30%
2. drop all rows with missing columns of more than 5


Function 3
Fill in Missing Values for numerical values, Null or Nan Values 
1. Use XGBoost and only numerical columns and encode columns variables that have unique values of less than 25 values to fill in missing/null/nan values 
2. Create a new csv in the data/processed folder named cleaneddata.csv (if it already exists replace it) that includes filled in missed values 


In [30]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

class DataCleaning:
    def __init__(self, parent_folder):
        self.parent_folder = parent_folder
        self.rawdata_path = os.path.join(self.parent_folder, 'data', 'processed', 'rawdata.csv')
        self.cleaneddata_path = os.path.join(self.parent_folder, 'data', 'processed', 'cleaneddata.csv')
        self.df = pd.read_csv(self.rawdata_path)

    def change_column_formats(self):
        # Change date-like columns
        date_columns = [col for col in self.df.columns if 'date' in col.lower()]
        for col in date_columns:
            try:
                self.df[col] = pd.to_datetime(self.df[col], errors='coerce')
                self.df['year'] = self.df[col].dt.year
                self.df['month'] = self.df[col].dt.month
            except Exception as e:
                print(f"Could not convert {col} to datetime: {e}")

        # Change numerical columns with more than 30% numerical values
        for col in self.df.select_dtypes(include=['object']).columns:
            try:
                num_values = pd.to_numeric(self.df[col].str.replace(',', '').str.strip(), errors='coerce')
                if num_values.notnull().mean() > 0.3:
                    self.df[col] = num_values
            except Exception as e:
                print(f"Could not convert {col} to numeric: {e}")

    def drop_missing_values(self):
        # Drop columns with more than 30% missing values
        self.df.dropna(thresh=len(self.df) * 0.7, axis=1, inplace=True)

        # Drop rows with more than 5 missing columns
        self.df.dropna(thresh=len(self.df.columns) - 5, axis=0, inplace=True)

    def fill_missing_values(self):
        # Drop columns with date or datetime format
        date_columns = [col for col in self.df.columns if 'date' in col.lower()]
        self.df.drop(columns=date_columns, inplace=True, errors='ignore')

        # Encode categorical variables
        for col in self.df.select_dtypes(include=['object']).columns:
            if self.df[col].nunique() <= 25:
                self.df[col] = LabelEncoder().fit_transform(self.df[col].astype(str))

        # Fill missing values using XGBoost
        missing_columns = self.df.columns[self.df.isnull().any()].tolist()
        for col in missing_columns:
            try:
                X = self.df.drop(columns=missing_columns)
                y = self.df[col]
                missing_mask = y.isnull()
                train_X = X.loc[~missing_mask]
                train_y = y.loc[~missing_mask]
                test_X = X.loc[missing_mask]

                reg = xgb.XGBRegressor(objective='reg:squarederror')
                reg.fit(train_X, train_y)
                self.df.loc[missing_mask, col] = reg.predict(test_X)
            except Exception as e:
                print(f"Could not fill missing values for {col}: {e}")

        # Save the cleaned data to a new CSV file
        self.df.to_csv(self.cleaneddata_path, index=False)



In [31]:
parent_folder = 'T:\Github Projects\data-science-practice\Most-Streamed-Spotify-Songs-2024'

data_cleaning = DataCleaning(parent_folder)

In [32]:

data_cleaning.change_column_formats()

In [33]:

data_cleaning.drop_missing_values()

In [34]:

data_cleaning.fill_missing_values()

Could not fill missing values for Spotify Streams: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Track: object, Album Name: object, Artist: object, ISRC: object
Could not fill missing values for Spotify Popularity: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Track: object, Album Name: object, Artist: object, ISRC: object
Could not fill missing values for YouTube Views: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Track: object, Album Name: object, Artist: object, ISRC: object
Could not fill missing values for YouTube Likes: DataFrame.dtypes for da